In [1]:
%cd ..

/home/wenli/car_dynamics


# Dyn-Learning

## Single-Env

In [16]:
import ray
import wandb
import torch
from env_utils import make_env
from modules.dynamics import DynamicsLearning, VanillaDynamics, EncDynamics
from jax import random
import jax
from datetime import datetime
import string
%load_ext autoreload
%autoreload 2
    
DEBUG=False

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
ray.init(ignore_reinit_error=True, local_mode=DEBUG)

print("device", jax.devices())

NUM_DATA = 1_000_000
# NUM_DATA = 1_0_000
DATA_CHUNK = 10000
DATA_DIR = './'
INPUT_DIMS = 11
OUTPUT_DIMS = 5
ACTION_DIMS = 2
REPR_DIMS = 2 
BATCH_SIZE = 10_000
ENV_DIMS=4

now = datetime.now()
FILE_NAME = f'{now.strftime("%Y")}-{now.strftime("%m")}-{now.strftime("%d")}-{now.strftime("%H:%M:%S")}'
DYN_MODEL_DIR = f'models/dyn_model/{FILE_NAME}/'


DYN_DATA_DIR = f'dataset/{FILE_NAME}'

def env_gen():
    # return make_env('point-wind-v0')
    # return make_env('point-reach-goal-multi-v3', wind_list=WIND_LIST)
    return make_env('car-base-single')


run = wandb.init(
    project="car-dynamics",
    config= {
        'mode': 'dynamics',
        'num_data': NUM_DATA,
        'dyn_model_dir': DYN_MODEL_DIR,
        'dyn_data_dir': DYN_DATA_DIR,
        'single_env': True,
    },
    mode = "disabled" if DEBUG else "online",
)

key1 = random.PRNGKey(123)
# dyn_model = EncDynamics(INPUT_DIMS, OUTPUT_DIMS, ACTION_DIMS, key1,
#         env_dims=ENV_DIMS, model_dir=DYN_MODEL_DIR,DEBUG=DEBUG,batch_size=BATCH_SIZE, repr_dims=REPR_DIMS)

fix_model = VanillaDynamics(INPUT_DIMS, OUTPUT_DIMS, ACTION_DIMS, 
                            key1, model_dir=DYN_MODEL_DIR,DEBUG=DEBUG,
                            batch_size=BATCH_SIZE, env_dims=ENV_DIMS)

fix_learner = DynamicsLearning(fix_model, env_gen, run, DEBUG=DEBUG)
fix_learner.learn(num_data = NUM_DATA, train_epoches=1000, evaluate_episodes=20, data_chunk=DATA_CHUNK, save_data_dir=DYN_DATA_DIR)


2023-10-21 18:11:59,837	INFO worker.py:1476 -- Calling ray.init() again after it has already been called.


device [gpu(id=0)]


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


evaluating/validation_error0,█▆█▃▆▃▃▃▄▃▂▂▂▄▁▁▂▁▃▁▂▂▂▃▄▁▂▁▂▂▂▁▁▂▁▁▁▂▁▁
evaluating/validation_error1,▇▄▁▃▄▃▃▄▅▄▄▅▅▆▆▇█▇▆▇▆▆▅▅▄▃▃▄▄▅▄▄▄▅▅▅▄▄▄▅
evaluating/validation_error2,█▄▄▂▂▂▁▂▁▂▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
evaluating/validation_error3,█▂▁▂▂▁▁▂▁▁▁▂▂▁▁▁▁▁▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
evaluating/validation_error4,█▄▄▄▄▃▃▃▂▂▂▂▂▃▂▂▁▂▂▂▁▁▂▂▂▁▁▂▁▁▁▂▂▁▁▁▁▂▁▁
testing/delta0,▃▃▅▅▃▃▃▂▃▄▅▃▅▃▃█▄▄▂▄▅▄▄▅▄▄▃▃▃▃▄▄▅▄▅▁▆▃▄▄
testing/delta1,▄▄█▆█▃▃▁▄▄▆▃▅▄▄▇▅▆▆▄▄▄▄▄▄▄▄▃▄▇▄▄▅▅▆▃▆▄▄▆
testing/delta2,▅▅▇▄█▅▅▁▄▄▄▆▄▄▄▂▄▆▅▅▅▄▄▄▄▅▅▅▅▃▄▄▅▄▃▅▅▅▅▃
testing/delta3,▂▂▁▃▃▂▂▄▂▂▅▁▄▃▃█▃▂▃▂▃▂▁▂▃▂▂▂▂▂▂▂▁▃▄▂▂▂▂▃
testing/delta4,▆▃▃▃█▇▃▇▅▅▃▄▇▇▄▃▃▄▃▆▆▃▄▃▃▆▇▃▇▅▃▃▆▃▃█▁▆▄▆
testing/deltaReal0,▃▃▅▅▃▃▃▂▃▄▅▃▅▃▃█▄▄▂▄▅▄▄▅▄▄▃▃▃▃▄▄▅▄▅▁▆▃▄▄


Output()

num data: 1000000, data_chunk: 10000


data size 1000000 cnt 1000000
[INFO] Create path:dataset/2023-10-21-18:11:59
[INFO] On_train_start, data size: 1000000


Output()

Output()

/home/wenli/miniforge3/envs/safeDPA/lib/python3.8/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


## Multi-Env

In [2]:
import ray
import wandb
import torch
from env_utils import make_env
from modules.dynamics import DynamicsLearning, VanillaDynamics, EncDynamics
from jax import random
import jax
from datetime import datetime
import string
%load_ext autoreload
%autoreload 2
    
DEBUG=False

In [3]:
jax.__version__

'0.4.13'

In [11]:
import os
directory = 'dataset/1123/data.npz'
paths = directory.split('/')
print(paths)
if not os.path.exists(os.path.join(paths[:-1])):
    os.makedirs(directory)

['dataset', '1123', 'data.npz']


In [15]:
ray.init(ignore_reinit_error=True, local_mode=DEBUG)

print("device", jax.devices())

TASK = 'stage-3'
NUM_DATA = 10_000_000
# NUM_DATA = 1_0_000
DATA_CHUNK = 10000
DATA_DIR = './'
INPUT_DIMS = 11
OUTPUT_DIMS = 5
ACTION_DIMS = 2
REPR_DIMS = 2 
BATCH_SIZE = 10_000
ENV_DIMS=4

now = datetime.now()
FILE_NAME = f'{now.strftime("%Y")}-{now.strftime("%m")}-{now.strftime("%d")}-{now.strftime("%H:%M:%S")}'
DYN_MODEL_DIR = f'models/dyn_model/{FILE_NAME}/'


DYN_DATA_DIR = f'dataset/{FILE_NAME}'

def env_gen():
    # return make_env('point-wind-v0')
    # return make_env('point-reach-goal-multi-v3', wind_list=WIND_LIST)
    return make_env('car-base')


run = wandb.init(
    project="car-dynamics",
    config= {
        'mode': 'dynamics',
        'num_data': NUM_DATA,
        'dyn_model_dir': DYN_MODEL_DIR,
        'dyn_data_dir': DYN_DATA_DIR,
    },
    mode = "disabled" if DEBUG else "online",
)

key1 = random.PRNGKey(123)
dyn_model = EncDynamics(INPUT_DIMS, OUTPUT_DIMS, ACTION_DIMS, key1,
        env_dims=ENV_DIMS, model_dir=DYN_MODEL_DIR,DEBUG=DEBUG,batch_size=BATCH_SIZE, repr_dims=REPR_DIMS)
# dyn_model = VanillaDynamics(INPUT_DIMS, OUTPUT_DIMS, ACTION_DIMS, 
#                                 key1, model_dir=DYN_MODEL_DIR,DEBUG=DEBUG,batch_size=BATCH_SIZE)
dyn_learner = DynamicsLearning(dyn_model, env_gen, run, DEBUG=DEBUG)
# dyn_learner.learn(num_data = NUM_DATA, evaluate_episodes=10, train_epoches=1000, )
dyn_learner.learn(evaluate_episodes=10, train_epoches=1000, num_data=NUM_DATA,data_chunk=DATA_CHUNK, save_data_dir=DYN_DATA_DIR)


2023-10-21 12:01:14,314	INFO worker.py:1476 -- Calling ray.init() again after it has already been called.


device [gpu(id=0)]


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Output()

num data: 10000000, data_chunk: 10000


data size 10000000 cnt 10000000
[INFO] Create path:dataset/2023-10-21-12:01:14
[INFO] On_train_start, data size: 10000000


Output()

Output()

/home/wenli/miniforge3/envs/safeDPA/lib/python3.8/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
